Import the libraries 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input,  Activation
from keras.layers import  GlobalMaxPool1D, Dropout
from keras import  optimizers, layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = stopwords.words('english')
import html
import unicodedata
import html
from tqdm.auto import tqdm
import re
import string

1: Uploading the dataset

In [ ]:
# upload the data from file 
train_df = pd.read_csv('train.csv')

In [ ]:
# Generating 10 samples from the file
train_df.sample(10, random_state = 1234)

In [ ]:
# if cond lable == toxic bering it 
train_df.loc[train_df['toxic'] == 1].sample(10, random_state=1234)

2: Cleaning the comments

In [ ]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))


def to_lowercase(text):
    return text.lower()



def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

### search for it 
def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)

def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]

#####
#def stem_words(words):
   # """Stem words in text"""
    #stemmer = PorterStemmer()
    #return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
    return word_tokenize(text)

def cleaning_text( text):
    text = remove_special_chars(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

In [ ]:
# apply cleaning in all comment_text in dataset مرور  (understand  lambda )
train_df['comment_text'] = train_df['comment_text'].apply(lambda x: cleaning_text(x))

3: preprocessing

In [ ]:
# cleaning comment in x 
X = train_df['comment_text'] 
# y conten the class for a sntences == lables
y = train_df[train_df.columns[2:]].values

In [ ]:
# initialize a TextVectorization layer
# output_sequence_length every sentance we take from the dataset we convert to 600 length
# output_mode to convert each single word to integer
vocab=20000
vectorizer = TextVectorization(max_tokens=vocab,
                               output_sequence_length=600,
                               output_mode='int')

In [ ]:
# apabt and teach vectorizer our vocabulary
vectorizer.adapt(X.values)

In [ ]:
# vectorizer the words , convert the words to int then save it in vectorized_text
vectorized_text = vectorizer(X.values)

4: Create the model

In [ ]:
#MCSHBAP -basic data pipeline generation set # Tow ways to instantiate the data pipline (from_tensor slices) or (list_file)
# pass through our data that just we created ( vectorized_text is the input features ) & ( y is the target lable)
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))# creating the dataset
dataset = dataset.cache()# subsequent data pre_processing
dataset = dataset.shuffle(160000)# how large our buffer needs to be 
dataset = dataset.batch(16)# have each batch represented  as a series of 16 samples 
dataset = dataset.prefetch(8) # helps bottlenecks
#to build data pipline

In [ ]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [ ]:
# Define parameters
filters = 150
kernel_size = 3
hidden_dims = 150


In [ ]:
#Create the Embedding Layer
model = Sequential()
# Create the embedding layer 
model.add(Embedding(vocab+1, 32))
#model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, activation = 'relu'))
model.add(MaxPooling1D())
model.add(Conv1D(filters, kernel_size, activation = 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation = 'sigmoid'))

# Display the model structure
model.summary() 

5: Training the model

In [ ]:
# loss = error function , optimize the model  
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# train the model , epochs the num of traning and its random value 
batch_size = 64
history = model.fit(train , epochs=70 , batch_size = batch_size , validation_data=val)

6: Preducation & Evaulation 

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    # X_true = comment in text 
    # y_true = label for these comment 
    # batch = display comment and lebal
    X_true,y_true = batch
    
    # Make a prediction 
    # inter the x_true = comment and see the preduct come with val
    yhat = model.predict(X_true)
    
    # read about it ***
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    # compare betwenn the result y_true which is the true one  and yhat which the predect one from the model 
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Save the model

In [ ]:
model.save('toxicityNLP_Project10.h5')

Loading the model

In [ ]:
model = tf.keras.models.load_model('toxicityNLP_Project10.h5')

7: Testing

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer(comment)
    results = model.predict(np.expand_dims(vectorized_comment,0))
    
    text = ''
    for idx, col in enumerate(train_df.columns[2:]):
        if results[0][idx]>0.5:  #the result of our predect if > 0.5 is toxic 
           text += '{}\n'.format(col)
    if text == '':
      return 'Your comment dose not have any toxic'
    else: 
      return 'Your comment is : \n '+text + '( You should choose your words carefully,please be kind ! )'

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                         outputs='text')

In [ ]:
# just to run our interfce  
interface.launch(share=True)

